In [2]:
# Importerer de bilbiotekene som trenges ikke rør
import pdfplumber     # Bruker dette for å åpne prislisten
import pandas as pd   # Brukes til å lage Excel arket
import re             # Vi Bruker RegEx for å gjenkjenne varenummer og priser

In [17]:
# Henter prislisten
pdf_path = "example.pdf" # SØRG FOR AT FILNAVNET MÅ VÆRE EKSAKT
rows = []

# Prisliste starter ikke på side 1, men side 7 i akkurat mitt eksempel 
# Prisliste slutter ikke på siste side
# Derfor definerer jeg antall sider
start_page = 0 # startside er på 7, start_page må være (ekte startisde - 1)
end_page = 7  # sluttside som den er (151 er siste som blir tatt med)

# Løsning for å importere alle sidene
with pdfplumber.open(pdf_path) as pdf:
    # Blar gjennom alle sidene i PDF, fra første til siste side
    for i in range(start_page, end_page):
        try: # Her skal vi teste å se om det er tekst på sidene, hvis det ikke er det hopper vi over siden
            text = pdf.pages[i].extract_text() # Henter ut teksten
            if not text:
                print(f"Tom side {i+1}") # Hvis det ikke er tekst på siden så hopper vi over
                continue

            for line in text.split("\n"): # Sjekker alle linjene på siden i PDFen
                # Prøv å finne mønsteret: varenr først, pris sist
                # Vi vet at alle varene har et varenummer som likner XX-XXXXX (hvor X er tall)
                # Etterfulgt
                match = re.match(r'^([0-9]{2}[-‐][0-9]{5})\s+(.+?)\s+([\d ]+)$', line.strip())
                if match:
                    artnr = match.group(1).replace("‐", "-")  # bytt ut "feil" bindestrek når vi først er i gang 
                    #(Lettere å søke på leverandørens sider med –, prislisten var feil)
                    desc = match.group(2).strip()  # Henter beskrivelsen
                    price = match.group(3).replace(" ", "") # Henter prisen
                    if price.isdigit():
                        rows.append((artnr, desc, int(price))) # Legger til siden
            print(f"✅ Side {i+1} behandlet") # Bekrefter at ingenting var feil, vi er ferdig med siden
        except Exception as e:
            print(f"❌ Feil på side {i+1}: {e}") # Viser feil på side tall X fordi den er tom

# Lag excel fil
df = pd.DataFrame(rows, columns=["Art.nr", "Description", "Net price (SEK)"])
df.to_excel("example_output1.xlsx", index=False)
df.head() # Vi viser de første linjene for å sjekke at vi er all good

✅ Side 1 behandlet
✅ Side 2 behandlet
✅ Side 3 behandlet
✅ Side 4 behandlet
✅ Side 5 behandlet
✅ Side 6 behandlet
✅ Side 7 behandlet


,Art.nr,Description,Net price (SEK)
0,01-00075,Produktbeskrivelse,460
1,01-00077,Produktbeskrivelse,69501
2,01-00085,Produktbeskrivelse,56001
3,01-00087,Produktbeskrivelse,96001
4,01-00095,Produktbeskrivelse,640


## Leverandørspesefikke Veilpriser Tilburdspriser og forhandlertilbud etc

In [18]:
pdf_path = "example.pdf" # Henter prislisten
rows = []

# Satt til 1 fordi jeg ikke vil røpe våres leverandøres dekningsbirdag, men må egentlig endres
small_goods = 1 
small_goods_offer = 1
large_goods = 1
large_goods_offer = 1


def extract_group(description):
    return description.split()[0] if description else ""

def round_to_5(n): # Vi avrunder prisene våre for å matche leverandørens prissettingsguide
    return int(round(n / 5.0)) * 5

# Vi fikk ikke veilpris lagt ved i denne PDF'en, så jeg estimerer de her ved hjelp av denne funksjonen
def beregn_veilpris(pris):
    return round_to_5(pris * large_goods) if pris < 1650 else round_to_5(pris * small_goods)

# Vi regner ut tilbudspisene for når vi kjører kampanjer
def beregn_tilbudspriser(net_price_mva):
    kostpris = net_price_mva / 1.25
    db = 0.37 if net_price_mva < 1650 else 0.30
    pris_u_mva = kostpris / (1 - db)
    pris_m_mva = pris_u_mva * 1.25
    pris_m_mva_rundet = round_to_5(pris_m_mva)
    pris_u_mva_rundet = int(pris_m_mva_rundet / 1.25)
    return pris_u_mva_rundet, pris_m_mva_rundet

# Samme som over
with pdfplumber.open(pdf_path) as pdf:
    for i in range(6, len(pdf.pages)):
        try:
            text = pdf.pages[i].extract_text()
            if not text:
                continue

            for line in text.split("\n"):
                match = re.match(r'^([0-9]{2}[-‐][0-9]{5})\s+(.+?)\s+([\d ]+)$', line.strip())
                if match:
                    artnr = match.group(1).replace("‐", "-")
                    desc = match.group(2).strip()
                    price = match.group(3).replace(" ", "")
                    if price.isdigit():
                        net_price = int(price)
                        dealer_price = int(net_price * 0.90)
                        veilpris = beregn_veilpris(net_price)
                        tilbud_u_mva, tilbud_m_mva = beregn_tilbudspriser(net_price)
                        group = extract_group(desc)
                        rows.append((
                            artnr, desc, net_price,
                            dealer_price, veilpris,
                            tilbud_u_mva, tilbud_m_mva, group
                        ))

            print(f"✅ Side {i+1} behandlet")
        except Exception as e:
            print(f"❌ Feil på side {i+1}: {e}")

df = pd.DataFrame(rows, columns=[
    "Art.nr", "Description", "Net price (SEK)",
    "Forhandlerpris (SEK)", "VEILPRIS (SEK)",
    "Tilbudspris u/mva (SEK)", "Tilbudspris m/mva (SEK)", "Group"
])

df.sort_values(by=["Group", "Art.nr"], inplace=True)

df.to_excel("example_output2.xlsx", index=False)

df.head()

✅ Side 7 behandlet
✅ Side 8 behandlet
✅ Side 9 behandlet
✅ Side 10 behandlet
✅ Side 11 behandlet
✅ Side 12 behandlet
✅ Side 13 behandlet


,Art.nr,Description,Net price (SEK),Forhandlerpris (SEK),VEILPRIS (SEK),Tilbudspris u/mva (SEK),Tilbudspris m/mva (SEK),Group
0,06-20319,Produktbeskrivelse,169506,152555,169505,193720,242150,Produktbeskrivelse
1,06-20619,Produktbeskrivelse,219506,197555,219505,250864,313580,Produktbeskrivelse
2,06-20819,Produktbeskrivelse,145506,130955,145505,166292,207865,Produktbeskrivelse
3,06-21019,Produktbeskrivelse,217506,195755,217505,248580,310725,Produktbeskrivelse
4,06-21319,Produktbeskrivelse,2,1,0,4,5,Produktbeskrivelse


# DEBUGGING

In [19]:
# Bare brukt for å se hvordan den ser ut for å hente ut informasjonen rett
with pdfplumber.open(pdf_path) as pdf:
    page = pdf.pages[6]
    print(page.extract_text())

06-20319 Produktbeskrivelse 1 69506
06-20619 Produktbeskrivelse 2 19506
06-20819 Produktbeskrivelse 1 45506
06-21019 Produktbeskrivelse 2 17506
06-21319 Produktbeskrivelse 2
06-21413 Produktbeskrivelse 6 64506
06-21813 Produktbeskrivelse 1 09506
06-22542 Produktbeskrivelse 80
06-22619 Produktbeskrivelse 2 52006
06-22719 Produktbeskrivelse 7 16006
06-22819 Produktbeskrivelse 4 88506
06-23019 Produktbeskrivelse 24506
06-23119 Produktbeskrivelse 15506
06-23419 Produktbeskrivelse 59506
06-23819 Produktbeskrivelse 54506
06-23919 Produktbeskrivelse 1 74506
06-24042 Produktbeskrivelse 3 89506
06-24219 Produktbeskrivelse 20006
06-24342 Produktbeskrivelse 14506
06-24442 Produktbeskrivelse 26506
06-24542 Produktbeskrivelse 3
06-24719 Produktbeskrivelse 3
06-24810 Produktbeskrivelse 3
06-24819 Produktbeskrivelse 3
06-24842 Produktbeskrivelse 3
06-25019 Produktbeskrivelse 1 89506
06-25119 Produktbeskrivelse 5 43506
06-25519 Produktbeskrivelse 21506
06-25819 Produktbeskrivelse 29506
06-25942 Produk